In [1]:
# unlike a user defined class Coordinate,
# this has a useful __repr__ and meaningful __eq__
from collections import namedtuple
Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [2]:
moscow = Coordinate(55.756, 37.617)
moscow

Coordinate(lat=55.756, lon=37.617)

In [4]:
moscow == Coordinate(lat=55.756, lon=37.617)

True

In [5]:
# or Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float) and allows for **fields
import typing
Coordinate = typing.NamedTuple('Coordinate', [('lat', float), ('lon', float)])
issubclass(Coordinate, tuple)

True

In [6]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [8]:
# NamedTuple is not a superclass in this case to Coordinate
# This uses the advanced functionality of a metaclass
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [16]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

    def __str__(self):
        ns = 'N' if self.lat >= 0 else 'S'
        we = 'E' if self.lon >= 0 else 'W'
        return f'{abs(self.lat):.1f}°{ns}, {abs(self.lon):.1f}°{we}'

In [17]:
# Named Tuple, many functions of the Python standard lib used to return tuples now return names tuples
# this doesn't affect the users code at all (better __repr__)
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [18]:
tokyo.population

36.933

In [19]:
tokyo.coordinates

(35.689722, 139.691667)

In [20]:
tokyo[1]

'JP'

In [21]:
City._fields

('name', 'country', 'population', 'coordinates')

In [22]:
Coordinates = namedtuple('Coordinate', 'lat lon')
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))
delhi = City._make(delhi_data)
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

In [69]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WGS84'])
Coordinate(0,0)

Coordinate(lat=0, lon=0, reference='WGS84')

In [70]:
Coordinate._field_defaults

{'reference': 'WGS84'}

In [71]:
# every instance field must be annotated,
# python does not enforce type hints at runtime! useful for an IDE running a linter
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float
    reference: str = 'WGS84'

In [72]:
"""
Classes built by typing.NamedTuple don’t have any methods beyond those that col
lections.namedtuple also generates—and those that are inherited from tuple. The
only difference is the presence of the __annotations__ class attribute—which Python
completely ignores at runtime.
"""
class DemoPlainClass:
    a: int # entry in __annotations__, otherwise discarded no attribute named a is created in the class
    b: float = 1.1 # saved as an annotation
    c = 'spam' # class attribute, no annotation

In [74]:
DemoPlainClass.__annotations__

{'a': int, 'b': float}

In [75]:
DemoPlainClass.a # of course python didn't make the same mistake as javascript (no concept as undefined)

AttributeError: type object 'DemoPlainClass' has no attribute 'a'

In [76]:
DemoPlainClass.b

1.1

In [77]:
DemoPlainClass.c

'spam'

In [79]:
class DemoNTClass(typing.NamedTuple):
    a: int
    b: float = 1.1
    c = 'spam'

In [80]:
DemoNTClass.__annotations__

{'a': int, 'b': float}

In [81]:
DemoNTClass.a

_tuplegetter(0, 'Alias for field number 0')

In [82]:
DemoNTClass.b

_tuplegetter(1, 'Alias for field number 1')

In [83]:
DemoNTClass.c

'spam'

In [84]:
DemoNTClass.__doc__

'DemoNTClass(a, b)'

In [85]:
nt = DemoNTClass(8)
nt.a

8

In [86]:
nt.b

1.1

In [87]:
nt.c

'spam'

In [88]:
@dataclass
class DemoDataClass:
    a: int
    b: float = 1.1
    c = 'spam'

In [89]:
DemoDataClass.__doc__

'DemoDataClass(a: int, b: float = 1.1)'

In [90]:
DemoDataClass.a

AttributeError: type object 'DemoDataClass' has no attribute 'a'

In [91]:
DemoDataClass.b

1.1

In [92]:
DemoDataClass.c

'spam'

In [93]:
dc = DemoDataClass(9)
dc.a

9

In [94]:
dc.b

1.1

In [95]:
dc.c

'spam'

In [96]:
dc.a = 10
dc.b = 'oops'

In [97]:
dc.c = 'whatever'
dc.z = 'secret stash'
dc

DemoDataClass(a=10, b='oops')

In [98]:
# if @dataclass are both frozen and eq to True it will set a __hash__
@dataclass
class ClubMember:
    name: str
    guests: list = [] # this will give an error

ValueError: mutable default <class 'list'> for field guests is not allowed: use default_factory

In [100]:
# this change allows us to invoke with zero arguments
# each instance will have its own list instead of all instances sharing the same list from the class
from dataclasses import dataclass, field
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)

In [102]:
# more precise
@dataclass
class ClubMember:
    name: str
    guests: list[str] = field(default_factory=list)

In [110]:
@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()
    handle: str = ''

    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)

In [111]:
anna = HackerClubMember('Anna Ravenscroft', handle='AnnaRaven')
anna

HackerClubMember(name='Anna Ravenscroft', guests=[], handle='AnnaRaven')

In [112]:
leo = HackerClubMember('Leo Rochael')
leo

HackerClubMember(name='Leo Rochael', guests=[], handle='Leo')

In [113]:
leo2 = HackerClubMember('Leo DaVinci') # should throw because leo already is in all_handles

ValueError: handle 'Leo' already exists.

In [126]:
# init only variables (in case we want to pass arguments to init that not instance fields
# commented out for example purposes, @dataclass supports this
from dataclasses import InitVar
@dataclass
class C:
    i: int
    j: int = None
    #database: InitVar[DatabaseType] = None

    def __post_init__(self, database):
        if self.j is None and database is not None:
            self.j = database.lookup('j')

#c = C(10, database=my_database)

In [141]:
# dublin core
from dataclasses import dataclass, fields
from typing import Optional
from enum import Enum, auto
from datetime import date

class ResourceType(Enum):
    BOOK = auto()
    EBOOK = auto()
    VIDEO = auto()

@dataclass
class Resource:
    identifier: str
    title: str = '<untitled>'
    creators: list[str] = field(default_factory=list)
    date: Optional[date] = None
    type: ResourceType = ResourceType.BOOK
    description: str = ''
    language: str = ''
    subjects: list[str] = field(default_factory=list)

    # added after to improve the output readability of our resource
    def __repr__(self):
        cls = self.__class__
        cls_name = cls.__name__
        indent = ' ' * 4
        res = [f'{cls_name}(']
        for f in fields(cls):
            value = getattr(self, f.name)
            res.append(f'{indent}{f.name} = {value!r},')

        res.append(')')
        return '\n'.join(res)

In [142]:
description = 'Improving the design of existing code'
book = Resource('978-0-13-475759-9', 'Refactoring, 2nd Edition', ['Martin Fowler', 'Kent Beck'], date(2018, 11, 19), ResourceType.BOOK, description, 'EN', ['computer programming', 'OOP'])
book

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

In [143]:
# more readable:
book = Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = date(2018, 11, 19),
    type = ResourceType.BOOK,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP']
)
book

Resource(
    identifier = '978-0-13-475759-9',
    title = 'Refactoring, 2nd Edition',
    creators = ['Martin Fowler', 'Kent Beck'],
    date = datetime.date(2018, 11, 19),
    type = <ResourceType.BOOK: 1>,
    description = 'Improving the design of existing code',
    language = 'EN',
    subjects = ['computer programming', 'OOP'],
)

In [144]:
# Keyword class patterns, positional class patterns
import typing

class City(typing.NamedTuple):
    continent: str
    name: str
    country: str

cities = [
    City('Asia', 'Tokyo', 'JP'),
    City('Asia', 'Delhi', 'IN'),
    City('North America', 'Mexico City', 'MX'),
    City('North America', 'New York', 'US'),
    City('South America', 'São Paulo', 'BR'),
]

def match_asian_cities():
    results = []
    for city in cities:
        match city:
            case City(continent='Asia', country=country):
                results.append(country)
    return results

def match_asian_cities_pos():
    results = []
    for city in cities:
        match city:
            case City('Asia'):
                results.append(city)
    return results

In [145]:
City.__match_args__

('continent', 'name', 'country')